<a href="https://colab.research.google.com/github/Naveensadanandan/Personal_fitness_coach/blob/main/Fitness_coach_project_RAGpipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document processing or embedding creation

In [1]:
import os
import requests

github = "https://raw.githubusercontent.com/Naveensadanandan/Personal_fitness_coach/main/data/"

file_names = ["ncert_textbook_fitness.pdf","human-nutrition-text.pdf","JN - UpperLower Program.pdf"]

save_dir = "Data/"

if not os.path.exists(save_dir):
  print(f"file path doesnt exist, downloading")
  os.makedirs(save_dir, exist_ok=True)

  for file_name in file_names:

    base_url = github + file_name

    response = requests.get(base_url)

    if response.status_code == 200:
      with open(os.path.join(save_dir,file_name),"wb") as f:
        f.write(response.content)
      print(f"file has been saved as {file_name}")
    else:
      print(f"[INFO] failed to downlaod : status code {response.status_code}")
else:
  print(f"Directory already exists, skipping download")

file path doesnt exist, downloading
file has been saved as ncert_textbook_fitness.pdf
file has been saved as human-nutrition-text.pdf
file has been saved as JN - UpperLower Program.pdf


In [2]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 26.5 MB/s eta 0:00:00


In [3]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text:str) -> str:
  cleaned_text = text.replace("\n"," ").strip()
  return cleaned_text

def read_write_pdf(filepath: str) -> list[dict]:
  combined_list = []
  for file_name in os.listdir(filepath):
    filepath_for_pdf = os.path.join(filepath,file_name)
    if os.path.isfile(filepath_for_pdf):
      doc = fitz.open(filepath_for_pdf)

      for page_number , page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        combined_list.append({"source" : file_name,
                                "page_number": page_number,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_setence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4, # 1 token = ~4 characters
                                "text": text})
  return combined_list

In [4]:
pages_and_texts = read_write_pdf(filepath=save_dir)
pages_and_texts[:2]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[{'source': 'ncert_textbook_fitness.pdf',
  'page_number': 0,
  'page_char_count': 95,
  'page_word_count': 19,
  'page_setence_count_raw': 1,
  'page_token_count': 23.75,
  'text': 'Health and Physical  Education Textbook for Class IX pre pages.indd   1 18-11-2016   4:25:29 PM'},
 {'source': 'ncert_textbook_fitness.pdf',
  'page_number': 1,
  'page_char_count': 1889,
  'page_word_count': 327,
  'page_setence_count_raw': 9,
  'page_token_count': 472.25,
  'text': 'First Edition November 2016 Kartika 1938 PD  5T BS ©\tNational Council of Educational  Research and Training, 2016 ` 135.00 Printed on 80 GSM paper with NCERT  watermark Published at the Publication Division  by the Secretary, National Council of  Educational Research and Training,   Sri Aurobindo Marg, New Delhi 110 016  and ................................... ISBN 978-93-5007-819-8 OFFICES OF THE PUBLICATION   DIVISION, NCERT NCERT Campus Sri Aurobindo Marg New  Delhi 110 016\t Phone :  011-26562708 108, 100 Feet Road  Hosd

In [5]:
import random
random.sample(pages_and_texts, k = 5)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 723,
  'page_char_count': 235,
  'page_word_count': 50,
  'page_setence_count_raw': 2,
  'page_token_count': 58.75,
  'text': 'Image by  Chris55 / CC  BY 4.0\xa0\xa0 A large  goiter by Dr.  J.S.Bhandari,  India / CC  BY-SA 3.0  Figure 11.6 Iodine Deficiency: Goiter  Dietary Reference Intakes for Iodine  Table 11.8 Dietary Reference Intakes for Iodine  682  |  Iodine'},
 {'source': 'human-nutrition-text.pdf',
  'page_number': 850,
  'page_char_count': 1750,
  'page_word_count': 261,
  'page_setence_count_raw': 12,
  'page_token_count': 437.5,
  'text': 'breastfeed. Once a new mother has left the hospital for home, she  needs access to a trained individual who can provide consistent  information. International Board Certified Lactation Consultants  (IBCLCs) are health-care professionals (often a registered nurse or  registered dietitian) certified in breastfeeding management that  work with new mothers to solve problems and educate

In [6]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df

,source,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,ncert_textbook_fitness.pdf,0,95,19,1,23.75,Health and Physical Education Textbook for Cl...
1,ncert_textbook_fitness.pdf,1,1889,327,9,472.25,First Edition November 2016 Kartika 1938 PD 5...
2,ncert_textbook_fitness.pdf,2,2187,364,17,546.75,"Foreword Many factors like, physical, psycholo..."
3,ncert_textbook_fitness.pdf,3,373,62,2,93.25,As an organisation committed to systemic refor...
4,ncert_textbook_fitness.pdf,4,902,129,8,225.50,Textbook Development Committee Members Bharati...
...,...,...,...,...,...,...,...
1506,human-nutrition-text.pdf,1203,1676,252,18,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...
1507,human-nutrition-text.pdf,1204,1617,254,10,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...
1508,human-nutrition-text.pdf,1205,1715,261,13,428.75,Flashcard Images Note: Most images in the fla...
1509,human-nutrition-text.pdf,1206,1733,268,13,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...


In [12]:
file_names = ["ncert_textbook_fitness.pdf","human-nutrition-text.pdf","JN - UpperLower Program.pdf"]
#since every document page number doesnt start with 1 we need to change page number for evaluation in the future.
page_start = {"ncert_textbook_fitness.pdf" : 12,
              "human-nutrition-text.pdf" : 41,
              "JN - UpperLower Program.pdf" : 0}
for key, value in page_start.items():
  print(key,value)

ncert_textbook_fitness.pdf 12
human-nutrition-text.pdf 41
JN - UpperLower Program.pdf 0


In [15]:
df_copy = df.copy()
for key, value in page_start.items():
  df_copy.loc[df_copy["source"] == key, "page_number"] = df_copy["page_number"] - value
df_copy

,source,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count,text
0,ncert_textbook_fitness.pdf,-12,95,19,1,23.75,Health and Physical Education Textbook for Cl...
1,ncert_textbook_fitness.pdf,-11,1889,327,9,472.25,First Edition November 2016 Kartika 1938 PD 5...
2,ncert_textbook_fitness.pdf,-10,2187,364,17,546.75,"Foreword Many factors like, physical, psycholo..."
3,ncert_textbook_fitness.pdf,-9,373,62,2,93.25,As an organisation committed to systemic refor...
4,ncert_textbook_fitness.pdf,-8,902,129,8,225.50,Textbook Development Committee Members Bharati...
...,...,...,...,...,...,...,...
1506,human-nutrition-text.pdf,1162,1676,252,18,419.00,39. Exercise 10.2 & 11.3 reused “Egg Oval Food...
1507,human-nutrition-text.pdf,1163,1617,254,10,404.25,Images / Pixabay License; “Pumpkin Cartoon Ora...
1508,human-nutrition-text.pdf,1164,1715,261,13,428.75,Flashcard Images Note: Most images in the fla...
1509,human-nutrition-text.pdf,1165,1733,268,13,433.25,ShareAlike 11. Organs reused “Pancreas Organ ...


In [16]:
df_copy.describe()

,page_number,page_char_count,page_word_count,page_setence_count_raw,page_token_count
count,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000
mean,465.829914,1269.018531,221.774322,11.028458,317.254633
std,367.792908,654.278173,116.622741,7.283101,163.569543
min,-41.000000,0.000000,1.000000,1.000000,0.000000
25%,118.500000,818.500000,141.000000,5.000000,204.625000
50%,411.000000,1327.000000,229.000000,11.000000,331.750000
75%,788.500000,1701.500000,293.500000,16.000000,425.375000
max,1166.000000,3452.000000,627.000000,47.000000,863.000000


# Using Spacy to sentensize our text

In [17]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")

In [18]:
pages_and_texts = df_copy.to_dict(orient = "records")
pages_and_texts

[{'source': 'ncert_textbook_fitness.pdf',
  'page_number': -12,
  'page_char_count': 95,
  'page_word_count': 19,
  'page_setence_count_raw': 1,
  'page_token_count': 23.75,
  'text': 'Health and Physical  Education Textbook for Class IX pre pages.indd   1 18-11-2016   4:25:29 PM'},
 {'source': 'ncert_textbook_fitness.pdf',
  'page_number': -11,
  'page_char_count': 1889,
  'page_word_count': 327,
  'page_setence_count_raw': 9,
  'page_token_count': 472.25,
  'text': 'First Edition November 2016 Kartika 1938 PD  5T BS ©\tNational Council of Educational  Research and Training, 2016 ` 135.00 Printed on 80 GSM paper with NCERT  watermark Published at the Publication Division  by the Secretary, National Council of  Educational Research and Training,   Sri Aurobindo Marg, New Delhi 110 016  and ................................... ISBN 978-93-5007-819-8 OFFICES OF THE PUBLICATION   DIVISION, NCERT NCERT Campus Sri Aurobindo Marg New  Delhi 110 016\t Phone :  011-26562708 108, 100 Feet Road  

In [20]:
for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)

  item["sentences"] = [str(sentense) for sentense in item["sentences"]]

  item["spacy_sentence_count"] = len(item["sentences"])

  0%|          | 0/1511 [00:00<?, ?it/s]

In [22]:
random.sample(pages_and_texts, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 333,
  'page_char_count': 1140,
  'page_word_count': 184,
  'page_setence_count_raw': 13,
  'page_token_count': 285.0,
  'text': '• HDL. Greater than 40–60 mg/dL  • triglycerides. 10–150 mg/dL  • VLDL. 2–38 mg/dL  Balancing Your Diet with Lipids  You may reason that if some fats are healthier than other fats, why  not consume as much healthy fat as desired? Remember, everything  in moderation. As we review the established guidelines for daily fat  intake, the importance of balancing fat consumption with proper fat  sources will be explained.  Recommended Fat Intake  The acceptable macronutrient distribution range (AMDR) from the  Dietary Reference Intake Committee for adult fat consumption is as  follows2:  • Fat calories should be limited to 20–35 percent of total calories  with most fats coming from polyunsaturated and  monounsaturated fats, such as those found in fish, nuts, and  vegetable oils.  • Consume fewer than 10 percen

In [24]:
from os.path import split
#chunking sentence togother
split_size_for_chunking = 10
def split_list(input_list : list,
               split_size : int = split_size_for_chunking) -> list[list]:
               chunk_list = [input_list[i:i+split_size] for i in range(0,len(input_list),split_size)]
               return chunk_list

test_list = list(range(25))

split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [25]:
for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_list(input_list = item["sentences"], split_size= split_size_for_chunking)
  item["chunk_count"] = len(item["sentence_chunks"])

  0%|          | 0/1511 [00:00<?, ?it/s]

In [37]:
random.sample(pages_and_texts, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 609,
  'page_char_count': 297,
  'page_word_count': 60,
  'page_setence_count_raw': 2,
  'page_token_count': 74.25,
  'text': 'Choy Sum by  pxhere.com  / CCO  Introduction  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  He pūko`a kani `āina  A coral reef strengthens into land.  Learning Objectives  By the end of this chapter you will be able to:  Introduction  |  609',
  'sentences': ['Choy Sum by  pxhere.com  / CCO  Introduction  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  He pūko`a kani `āina  A coral reef strengthens into land.',
   ' Learning Objectives  By the end of this chapter you will be able to:  Introduction  |  609'],
  'spacy_sentence_count': 2,
  'sentence_chunks': [['Choy Sum by  pxhere.com  / CCO  Introduction  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION 

## Creating an attribute dictionary for each chunks

In [42]:
import re

chunks = []
for item in tqdm(pages_and_texts):
  for chunk in item["sentence_chunks"]:
    chunk_dict = {}
    chunk_dict["source"] = item["source"]
    chunk_dict["page_number"] = item["page_number"]

    joined_sentence = "".join(chunk).replace("  "," ").strip()
    joined_sentence = re.sub(r'\.([A-Z])', r'. \1', joined_sentence)

    chunk_dict["sentence_chunk"] = joined_sentence

    # Get some stats on our chunks
    chunk_dict["chunk_char_count"] = len(joined_sentence)
    chunk_dict["chunk_word_count"] = len([word for word in joined_sentence.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence) / 4 # 1 token = ~4 chars

    chunks.append(chunk_dict)

  0%|          | 0/1511 [00:00<?, ?it/s]

In [44]:
random.sample(chunks, k=1)

[{'source': 'human-nutrition-text.pdf',
  'page_number': 623,
  'sentence_chunk': 'Consuming more calcium than is recommended is not better for your health and can prove to be detrimental. Consuming too much calcium at any one time, be it from diet or supplements, impairs not only the absorption of calcium itself, but also the absorption of other essential minerals, such as iron and zinc. Since the GI tract can only handle about 500 milligrams of calcium at one time, it is recommended to have split doses of calcium supplements rather than taking a few all 10.\xa0Bolland MJ.et al. (2010). Effect of Calcium Supplements on Risk of Myocardial Infarction and Cardiovascular Events: Meta-Analysis. British Medical Journal,\xa0341(c3691). Calcium | 623',
  'chunk_char_count': 664,
  'chunk_word_count': 107,
  'chunk_token_count': 166.0}]

In [45]:
df = pd.DataFrame(chunks)
df.head()

,source,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,ncert_textbook_fitness.pdf,-12,Health and Physical Education Textbook for Cla...,92,16,23.00
1,ncert_textbook_fitness.pdf,-11,First Edition November 2016 Kartika 1938 PD 5T...,1858,296,464.50
2,ncert_textbook_fitness.pdf,-10,"Foreword Many factors like, physical, psycholo...",1543,235,385.75
3,ncert_textbook_fitness.pdf,-10,Experiential learning activities for acquiring...,605,91,151.25
4,ncert_textbook_fitness.pdf,-9,As an organisation committed to systemic refor...,367,56,91.75


In [46]:
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,2522.00,2522.00,2522.00,2522.00
mean,450.75,742.84,115.85,185.71
std,370.34,448.76,72.68,112.19
min,-41.00,12.00,3.00,3.00
25%,115.00,347.00,52.00,86.75
50%,354.50,739.00,115.00,184.75
75%,788.75,1097.00,172.00,274.25
max,1166.00,2353.00,357.00,588.25
